In [7]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import joblib

In [ ]:
class ATMRefillOptimizer:
    def __init__(self, model_path, atm_capacity, safety_buffer):
        self.model = joblib.load(model_path)
        self.min_cash_threshold = 1000
        self.conversation_log = []
        self.atm_capacity = atm_capacity  # Maximum cash capacity of ATM
        self.safety_buffer = safety_buffer  # Safety buffer amount

    def get_time_period(self, hour):
        if 6 <= hour < 12: return 'Morning'
        elif 12 <= hour < 18: return 'Afternoon'
        elif 18 <= hour < 22: return 'Evening'
        else: return 'Night'

    def collect_essential_inputs_only(self):
        current_date = datetime.now()
        self.conversation_log.append("=== Essential ATM Information ===")

        inputs = {}

        # Auto-filled features
        inputs['Day_of_Week'] = current_date.strftime('%A')
        inputs['Time_of_Day'] = self.get_time_period(current_date.hour)
        self.conversation_log.append(f"Day of week: {inputs['Day_of_Week']}")
        self.conversation_log.append(f"Time of day: {inputs['Time_of_Day']}")

        # Core required inputs with logging
        current_cash = float(input("Current cash level: $"))
        inputs['Previous_Day_Cash_Level'] = current_cash
        self.conversation_log.append(f"Current cash level: ${current_cash:,.0f}")

        withdrawals = float(input("Yesterday's withdrawals: $"))
        inputs['Total_Withdrawals'] = withdrawals
        self.conversation_log.append(f"Yesterday's withdrawals: ${withdrawals:,.0f}")

        deposits = float(input("Yesterday's deposits: $"))
        inputs['Total_Deposits'] = deposits
        self.conversation_log.append(f"Yesterday's deposits: ${deposits:,.0f}")

        location = input("Location (Bank Branch/Mall/Gas Station/Standalone): ")
        inputs['Location_Type'] = location
        self.conversation_log.append(f"Location type: {location}")

        # Contextual inputs
        holiday_input = input("Holiday? (y/n): ").lower()
        inputs['Holiday_Flag'] = 1 if holiday_input == 'y' else 0
        self.conversation_log.append(f"Holiday: {'Yes' if inputs['Holiday_Flag'] else 'No'}")

        special_event_input = input("Special event nearby (concert, festival, etc.)? (y/n): ").lower()
        inputs['Special_Event_Flag'] = 1 if special_event_input == 'y' else 0
        self.conversation_log.append(f"Special event: {'Yes' if inputs['Special_Event_Flag'] else 'No'}")

        weather = input("Weather (Clear/Rainy/Cloudy): ") or 'Clear'
        inputs['Weather_Condition'] = weather
        self.conversation_log.append(f"Weather condition: {weather}")

        nearby_atms = int(input("Nearby ATMs (0-10): ") or 3)
        inputs['Nearby_Competitor_ATMs'] = nearby_atms
        self.conversation_log.append(f"Nearby competitor ATMs: {nearby_atms}")

        demand_2_days = float(input("Cash demand 2 days ago (or recent avg): $"))
        inputs['Cash_Demand_Lag_2'] = demand_2_days
        self.conversation_log.append(f"Cash demand 2 days ago: ${demand_2_days:,.0f}")

        # 7-day lag - optional
        demand_7_days_input = input("Cash demand 7 days ago (press Enter to skip): $")
        if demand_7_days_input.strip():
            inputs['Cash_Demand_Lag_7'] = float(demand_7_days_input)
            self.conversation_log.append(f"Cash demand 7 days ago: ${inputs['Cash_Demand_Lag_7']:,.0f}")
        else:
            inputs['Cash_Demand_Lag_7'] = np.nan
            self.conversation_log.append("Cash demand 7 days ago: Not provided (using model default)")

        # 3-day moving average - optional
        ma_3_input = input("Average cash demand last 3 days (press Enter to skip): $")
        if ma_3_input.strip():
            inputs['Cash_Demand_MA_3'] = float(ma_3_input)
            self.conversation_log.append(f"3-day average cash demand: ${inputs['Cash_Demand_MA_3']:,.0f}")
        else:
            inputs['Cash_Demand_MA_3'] = np.nan
            self.conversation_log.append("3-day average cash demand: Not provided (using model default)")

        # 7-day moving average - optional
        ma_7_input = input("Average cash demand last 7 days (press Enter to skip): $")
        if ma_7_input.strip():
            inputs['Cash_Demand_MA_7'] = float(ma_7_input)
            self.conversation_log.append(f"7-day average cash demand: ${inputs['Cash_Demand_MA_7']:,.0f}")
        else:
            inputs['Cash_Demand_MA_7'] = np.nan
            self.conversation_log.append("7-day average cash demand: Not provided (using model default)")

        # 7-day withdrawal average - optional
        withdrawals_ma_7_input = input("Average withdrawals last 7 days (press Enter to skip): $")
        if withdrawals_ma_7_input.strip():
            inputs['Withdrawals_MA_7'] = float(withdrawals_ma_7_input)
            self.conversation_log.append(f"7-day average withdrawals: ${inputs['Withdrawals_MA_7']:,.0f}")
        else:
            inputs['Withdrawals_MA_7'] = np.nan
            self.conversation_log.append("7-day average withdrawals: Not provided (using model default)")


        # Calculate derived features
        inputs['Net_Cash_Flow'] = deposits - withdrawals
        inputs['Withdrawal_to_Deposit_Ratio'] = withdrawals / (deposits + 1)
        inputs['Cash_Level_Change'] = 0
        inputs['Cash_Utilization_Rate'] = withdrawals / current_cash

        # Log calculated features
        self.conversation_log.append(f"Net cash flow: ${inputs['Net_Cash_Flow']:,.0f}")
        self.conversation_log.append(f"Cash utilization rate: {inputs['Cash_Utilization_Rate']:.1%}")

        return inputs

    def get_refill_recommendation(self):
        """Complete workflow with conversation logging"""
        
        # Collect inputs and log conversation
        inputs = self.collect_essential_inputs_only()
        
        # Create DataFrame for prediction
        input_df = pd.DataFrame([inputs])
        
        # Predict next day's cash demand
        predicted_demand = self.model.predict(input_df)[0]
        
        # Calculate refill recommendation
        current_cash = inputs['Previous_Day_Cash_Level']
        
        self.conversation_log.append("\n=== Analysis Results ===")
        self.conversation_log.append(f"Predicted tomorrow's demand: ${predicted_demand:,.0f}")
        self.conversation_log.append(f"Current cash level: ${current_cash:,.0f}")
        self.conversation_log.append(f"Cash after tomorrow: ${current_cash - predicted_demand:,.0f}")
        
        # Determine refill recommendation
        if current_cash - predicted_demand < self.min_cash_threshold:
            refill_day = 0
            recommendation = "Refill TODAY - cash will drop below threshold tomorrow"
        elif current_cash - (2 * predicted_demand) < self.min_cash_threshold:
            refill_day = 1
            recommendation = "Refill TOMORROW - will need cash within 2 days"
        else:
            days_until_refill = int((current_cash - self.min_cash_threshold) // predicted_demand)
            refill_day = max(0, days_until_refill - 1)
            recommendation = f"Refill in {refill_day + 1} days"
        
        self.conversation_log.append(f"Minimum cash threshold: ${self.min_cash_threshold:,.0f}")
        self.conversation_log.append(f"Recommendation: {recommendation}")
        
        return {
            'predicted_next_day_demand': predicted_demand,
            'current_cash': current_cash,
            'refill_day': refill_day,
            'recommendation': recommendation,
            'conversation_log': self.conversation_log.copy()
        }

    def print_conversation(self):
        """Print the complete conversation log"""
        print("\n" + "="*50)
        print("COMPLETE CONVERSATION LOG")
        print("="*50)
        for line in self.conversation_log:
            print(line)
        print("="*50 + "\n")
    
    def calculate_optimal_refill_amount(self, predicted_daily_demand, days_until_next_refill, current_cash_level):
        # Calculate planned fill amount
        total_demand_period = predicted_daily_demand * days_until_next_refill
        planned_fill = total_demand_period + self.safety_buffer - current_cash_level
        
        # Apply constraints: cannot exceed ATM capacity, cannot be negative
        optimal_fill = min(self.atm_capacity, max(0, planned_fill))
        
        # Log calculation details
        self.conversation_log.append("\n=== Optimal Refill Amount Calculation ===")
        self.conversation_log.append(f"Predicted daily demand: ${predicted_daily_demand:,.0f}")
        self.conversation_log.append(f"Days until next refill: {days_until_next_refill}")
        self.conversation_log.append(f"Total demand for period: ${total_demand_period:,.0f}")
        self.conversation_log.append(f"Safety buffer: ${self.safety_buffer:,.0f}")
        self.conversation_log.append(f"Current cash level: ${current_cash_level:,.0f}")
        self.conversation_log.append(f"ATM capacity: ${self.atm_capacity:,.0f}")
        self.conversation_log.append(f"Calculated fill before constraints: ${planned_fill:,.0f}")
        self.conversation_log.append(f"Optimal refill amount: ${optimal_fill:,.0f}")
        
        if planned_fill > self.atm_capacity:
            self.conversation_log.append("⚠️  Note: Refill amount capped by ATM capacity")
        if planned_fill < 0:
            self.conversation_log.append("ℹ️  Note: No refill needed (sufficient cash available)")
            
        return optimal_fill
    
    def get_refill_recommendation_with_amount(self):
        # Get basic refill recommendation
        basic_result = self.get_refill_recommendation()
        
        # Extract values for optimal amount calculation
        predicted_demand = basic_result['predicted_next_day_demand']
        current_cash = basic_result['current_cash']
        refill_day = basic_result['refill_day']
        
        # Determine days until next refill (minimum 1 day for planning purposes)
        days_until_refill = max(1, refill_day + 1)
        
        # Calculate optimal refill amount
        optimal_amount = self.calculate_optimal_refill_amount(
            predicted_daily_demand=predicted_demand,
            days_until_next_refill=days_until_refill,
            current_cash_level=current_cash
        )
        
        # Add optimal amount to results
        enhanced_result = basic_result.copy()
        enhanced_result['optimal_refill_amount'] = optimal_amount
        enhanced_result['days_until_refill'] = days_until_refill
        enhanced_result['conversation_log'] = self.conversation_log.copy()
        
        return enhanced_result
27



In [9]:
# Initialize with ATM capacity and safety buffer
optimizer = ATMRefillOptimizer(
    model_path='atm_cash_demand_model_linearregression.joblib',
    atm_capacity=75000,  # $50,000 ATM capacity
    safety_buffer=5000   # $5,000 safety buffer
)

# Get recommendation with optimal refill amount
result = optimizer.get_refill_recommendation_with_amount()

# Print the complete conversation
optimizer.print_conversation()

# Print enhanced results
print(f"\nPredicted tomorrow's demand: ${result['predicted_next_day_demand']:,.0f}")
print(f"Days until refill: {result['days_until_refill']}")
print(f"Optimal refill amount: ${result['optimal_refill_amount']:,.0f}")
print(f"Recommendation: {result['recommendation']}")



COMPLETE CONVERSATION LOG
=== Essential ATM Information ===
Day of week: Sunday
Time of day: Afternoon
Current cash level: $30,000
Yesterday's withdrawals: $25,000
Yesterday's deposits: $5,000
Location type: Mall
Holiday: No
Special event: Yes
Weather condition: Rainy
Nearby competitor ATMs: 3
Cash demand 2 days ago: $27,000
Cash demand 7 days ago: $32,000
3-day average cash demand: $28,000
7-day average cash demand: $30,000
7-day average withdrawals: $26,000
Net cash flow: $-20,000
Cash utilization rate: 83.3%

=== Analysis Results ===
Predicted tomorrow's demand: $33,329
Current cash level: $30,000
Cash after tomorrow: $-3,329
Minimum cash threshold: $1,000
Recommendation: Refill TODAY - cash will drop below threshold tomorrow

=== Optimal Refill Amount Calculation ===
Predicted daily demand: $33,329
Days until next refill: 1
Total demand for period: $33,329
Safety buffer: $5,000
Current cash level: $30,000
ATM capacity: $75,000
Calculated fill before constraints: $8,329
Optimal ref

Optimal Refill = min(ATM Capacity − Current Cash Level, (Predicted Demand for Period + Safety Buffer) − Current Cash Level)